In [0]:
# Paso 1: Leer el archivo CSV en un DataFrame
file_path = "/FileStore/tables/Mall_Customers-6.csv"  # Ajusta la ruta si es necesario

df = spark.read.csv(file_path, header=True, inferSchema=True)

# Paso 2: Crear una vista temporal
df.createOrReplaceTempView("mall_customers_view")


In [0]:
# Paso 1: Renombrar columnas
df = df.withColumnRenamed("Annual Income (k$)", "Annual_Income_k") \
       .withColumnRenamed("Spending Score (1-100)", "Spending_Score")

# Paso 2: Crear la vista temporal con las columnas renombradas
df.createOrReplaceTempView("mall_customers_view")

# Consulta 2 corregida: Obtener los clientes con un ingreso anual mayor a 70,000
spark.sql("SELECT * FROM mall_customers_view WHERE Annual_Income_k > 70").show()


+----------+------+---+---------------+--------------+
|CustomerID| Genre|Age|Annual_Income_k|Spending_Score|
+----------+------+---+---------------+--------------+
|       127|  Male| 43|             71|            35|
|       128|  Male| 40|             71|            95|
|       129|  Male| 59|             71|            11|
|       130|  Male| 38|             71|            75|
|       131|  Male| 47|             71|             9|
|       132|  Male| 39|             71|            75|
|       133|Female| 25|             72|            34|
|       134|Female| 31|             72|            71|
|       135|  Male| 20|             73|             5|
|       136|Female| 29|             73|            88|
|       137|Female| 44|             73|             7|
|       138|  Male| 32|             73|            73|
|       139|  Male| 19|             74|            10|
|       140|Female| 35|             74|            72|
|       141|Female| 57|             75|             5|
|       14

In [0]:
# Crear una tabla temporal persistente desde la vista temporal
spark.sql("CREATE OR REPLACE TEMPORARY VIEW mall_customers_temp AS SELECT * FROM mall_customers_view")

# Verificar que la tabla temporal se haya creado correctamente
spark.sql("SELECT * FROM mall_customers_temp").show()


+----------+------+---+---------------+--------------+
|CustomerID| Genre|Age|Annual_Income_k|Spending_Score|
+----------+------+---+---------------+--------------+
|         1|  Male| 19|             15|            39|
|         2|  Male| 21|             15|            81|
|         3|Female| 20|             16|             6|
|         4|Female| 23|             16|            77|
|         5|Female| 31|             17|            40|
|         6|Female| 22|             17|            76|
|         7|Female| 35|             18|             6|
|         8|Female| 23|             18|            94|
|         9|  Male| 64|             19|             3|
|        10|Female| 30|             19|            72|
|        11|  Male| 67|             19|            14|
|        12|Female| 35|             19|            99|
|        13|Female| 58|             20|            15|
|        14|Female| 24|             20|            77|
|        15|  Male| 37|             20|            13|
|        1

In [0]:
# Consulta para almacenar los resultados: Promedio del puntaje de gasto por género
result_df = spark.sql("SELECT Genre, AVG(Spending_Score) AS avg_spending FROM mall_customers_view GROUP BY Genre")

# Guardar el resultado como archivo Parquet
result_df.write.mode("overwrite").parquet("/FileStore/output/avg_spending_by_genre.parquet")


In [0]:
%sql
SELECT COUNT(*) AS total_customers FROM mall_customers_view;


total_customers
200


In [0]:
%sql
SELECT * FROM mall_customers_view WHERE Annual_Income_k > 70;


CustomerID,Genre,Age,Annual_Income_k,Spending_Score
127,Male,43,71,35
128,Male,40,71,95
129,Male,59,71,11
130,Male,38,71,75
131,Male,47,71,9
132,Male,39,71,75
133,Female,25,72,34
134,Female,31,72,71
135,Male,20,73,5
136,Female,29,73,88


In [0]:
%sql
SELECT Genre, COUNT(*) AS gender_count FROM mall_customers_view GROUP BY Genre;


Genre,gender_count
Female,112
Male,88


In [0]:
%sql
SELECT Genre, AVG(Spending_Score) AS avg_spending FROM mall_customers_view GROUP BY Genre;


Genre,avg_spending
Female,51.526785714285715
Male,48.51136363636363
